In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson

In [2]:
def getResponse(url, param = {}):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers, params=param)
    return response

In [3]:
# 获取页面的html文件
def getHTML(url):
    return getResponse(url).text

In [4]:
# 通过音频源地址爬取音频
def getAudio(url, id):
    response = getResponse(url)
    path = 'm4a/'
    #fileName = url.split('/')[-1]
    with open(path + id + '.m4a', 'wb') as fp:
        fp.write(response.content)

In [5]:
# 获取语音对应的文本
def getAudioText(html):    
    ex = '<article class="intro  eX_">(.*?)</article>'
    audio_text = re.findall(ex, html, re.S)
    ex = 'data-flag="normal">(.*?)</p>'
    audio_text = re.findall(ex, audio_text[0], re.S)
    text = "\n".join(audio_text)
    return text

In [6]:
# 获取音频源地址
# 通过get请求 返回的json数据中得到音频地址
# https://www.ximalaya.com/revision/play/v1/audio?id=148921429&ptype=1
def getAudioSrcUrl(id):
    url = 'https://www.ximalaya.com/revision/play/v1/audio'
    param = {
        'id': id,
        'ptype': '1',
    }
    response = getResponse(url=url, param=param)
    dic_obj = response.json()
    return dic_obj['data']['src']

In [7]:
def getIds(url):
    html = getHTML(url)
    tree = etree.HTML(html)
    allAudio = tree.xpath('//*[@id="anchor_sound_list"]/div[2]/ul/li')
    #print(len(allAudio))
    idList = []
    for audio in allAudio:
        srcUrl = audio.xpath('.//a/@href')[0]
        #print(srcUrl)
        idList.append(srcUrl.split('/')[-1])
    return idList

In [8]:
def getAllBook(url):
    html = getHTML(url)
    tree = etree.HTML(html)
    allBook = tree.xpath('//*[@id="award"]/main/div[1]/div/div[3]/div[1]/div/div[2]/ul/li')
    linkList = []
    # https://www.ximalaya.com/youshengshu/20322961/p2/
    for book in allBook:
        temp = book.xpath('.//a/@href')[0]
        #print(temp)
        linkList.append('https://www.ximalaya.com' + temp)
    return linkList

In [9]:
def downloadAudio(path, url):
    # https://aod.cos.tx.xmcdn.com/storages/d16e-audiofreehighqps/99/BC/CKwRINsEQiW5AEg_PQCbX6Rr.m4a
    print("音频", url)
    try:
        fileName = url.split('/')[-1]
        content = getResponse(url).content
        with open(path + '/' + fileName, 'wb') as fp:
            fp.write(content)
        print(fileName, "下载完毕")
    except Exception as e:
        print('error')

In [10]:
def downloadBook(url):
    # 如果一本书是付费的跳过
    print('download book', url)
    html = getHTML(url)
    # price-btn false _CO
    ex = '<article class="intro  eX_">(.*?)</article>'
    ex = 'price-btn false _CO'
    res = re.findall(ex, html, re.S)
    if len(res) > 0:
        print('付费')
        return
    
    # 创建该书的文件夹
    path = 'D:/work/商业非有声书3/' + url.split('/')[-2]
    if os.path.exists(path):
        print('已下载')
        return
    os.mkdir(path)
    i = 1
    while True:
        pageUrl = url + f'p{i}'
        print(pageUrl)
        ids = getIds(pageUrl)
        if len(ids) == 0:
            break
        i = i + 1
        for ID in ids:
            srcUrl = getAudioSrcUrl(ID)
            downloadAudio(path, srcUrl)
    print(url, "该书下载完毕")

In [11]:
# https://www.ximalaya.com/shangye/p1
for i in range(3, 6):
    print(f'开始下载 主页 {i}')
    url = f'https://www.ximalaya.com/shangye/p{i}'
    j = 0
    for book in getAllBook(url):
        if i == 3 and j < 6:
            continue
        j = j + 1
        print(f'book num {j}')
        downloadBook(book)

开始下载 主页 3
开始下载 主页 4
book num 1
download book https://www.ximalaya.com/shangye/3226253/
已下载
book num 2
download book https://www.ximalaya.com/shangye/29222395/
已下载
book num 3
download book https://www.ximalaya.com/shangye/26360817/
已下载
book num 4
download book https://www.ximalaya.com/shangye/47874894/
已下载
book num 5
download book https://www.ximalaya.com/shangye/36448943/
已下载
book num 6
download book https://www.ximalaya.com/shangye/14762063/
已下载
book num 7
download book https://www.ximalaya.com/shangye/11997832/
已下载
book num 8
download book https://www.ximalaya.com/shangye/47752492/
已下载
book num 9
download book https://www.ximalaya.com/renwen/43165567/
付费
book num 10
download book https://www.ximalaya.com/shangye/20498134/
已下载
book num 11
download book https://www.ximalaya.com/shangye/21701185/
已下载
book num 12
download book https://www.ximalaya.com/shangye/31103666/
已下载
book num 13
download book https://www.ximalaya.com/shangye/32445654/
已下载
book num 14
download book https://www.ximal